## Aggregate Results

### Self-Truncation

In [ ]:
# 1. check outputsteps
import glob

# folder = 'editv1'
folder = 'editv2'
# folder = 'sample64'
# folder = 'sample64_v2'
# folder = 'selftrunc64'
# folder = 'selftrunc64_v2'

# result_file_list = glob.glob(f"results/{folder}/gsm8k*")
result_file_list = glob.glob(f"results/{folder}/math500*")
# result_file_list = glob.glob(f"results/{folder}/aimo*")
# result_file_list = glob.glob(f"results/{folder}/aimo_r1*")
result_file_list

### EDIT (Dual-Goal Search)

In [ ]:
import json
import pandas as pd
import re, os
from mapping import modelname2hfpath
from collections import Counter
from utils import extract_last_boxed
from transformers import AutoTokenizer
from collections import defaultdict
from tqdm import tqdm

"""
{
    "<id>": {
        "question": str,
        "result_list": [
            {
                "output_solution": [
                    {'text': solution 1, 'cum_logprob': float},
                    {'text': solution 2, 'cum_logprob': float}
                    ...
                ],
                "max_steps": int
            }   # solutions and metadata at constriant 1
            {
                "output_solution": [
                    solution 1,
                    solution 2,
                ],
                "max_steps": int
            }   # solution at constriant 2
        ]
        "pred_num": int,
        "answer": str,
    }
}
"""

for res_idx in range(len(result_file_list)):
# for res_idx in range(2):
    filename = os.path.basename(result_file_list[res_idx])
    metadata = filename.replace('.jsonl', '').replace('.json', '').split("_")
    dataset, modelname = metadata[0], metadata[1]
    # if modelname not in ["r1-distill-llama8b-instruct"]:
        # continue
    tknz = AutoTokenizer.from_pretrained(modelname2hfpath[modelname])
    total_length = 0
    total_length_pen = 0
    correct = total = 0

    # res_df = pd.read_json(result_file_list[res_idx], lines=True)
    # sample 64 and self_trunc
    lines = False
    if filename.endswith('jsonl'):
        lines = True
    res_df = pd.read_json(result_file_list[res_idx], lines=lines)

    res_dict = {}
    cand_budget = [2048, 16384]
    for b in cand_budget:
        res_dict[b] = 0

    for _, group_rows in tqdm(res_df.groupby('id')):
        added = {k: False for k in cand_budget}
        for idx, row in group_rows.iterrows():
            per_step_solutions = row['output_solution'][:64]
            if dataset == "gsm8k":
                answer = re.findall(r"#### (-?\d+)", row["answer"])[-1]
            else:
                answer = row["answer"]
            
            for b in cand_budget:
                if added[b]:    # already added, no need to exam
                    continue
                cnter = Counter()
                answer_idx = defaultdict(list)

                for sol_idx, solution in enumerate(per_step_solutions):
                    # truncate
                    if isinstance(solution, dict):
                        solution = solution['text']
                    solution_tok = tknz.encode(solution)[:b]
                    solution = tknz.decode(solution_tok)

                    pred_answer = extract_last_boxed(solution)
                    if pred_answer is None:
                        continue
                    cnter[pred_answer] += 1
            
                if len(cnter) > 0:
                    sc_answer = cnter.most_common(1)[0][0]
                else:
                    sc_answer = None
                
                if sc_answer == str(answer):
                    res_dict[b] += 1
                    added[b] = True

        total += 1

    print(f"\nModel: {modelname}")
    for b, correct in res_dict.items():
        print(f"budget: {b}, correct: {correct}, total: {total}\n correct rate: {correct / total*100:.2f}%")